In [30]:
import numpy as np
import pandas as pd
import keras as keras

In [31]:
from keras.datasets import mnist
(train_X, train_y), (test_X, test_y) = mnist.load_data()

In [32]:
train_X.shape

(60000, 28, 28)

In [33]:
type(train_X)

numpy.ndarray

In [34]:
train_y

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

In [35]:
28*28


784

In [36]:
train = []
for i in range(len(train_X)):
  train.append(np.reshape(train_X[i], (1, 784)))

x_train = np.array(train)
y_train = train_y

In [37]:
y_train = np.reshape(train_y, (len(train_y)))

In [ ]:
# Preprocessing
for i in range(len(x_train)):
  for j in range(len(x_train[i][0])):
    if x_train[i][0][j] > 0 :
      x_train[i][0][j] = 1

In [ ]:
x_train[0].shape

In [ ]:
x_train[0]

In [ ]:
# Input neurons = 784
# Output neurons = 10

In [ ]:
y_train[0]

In [ ]:
# Process y
new_y_train = []
for i in range(len(y_train)):
  l = np.zeros((10))
  l[y_train[i]]=1
  new_y_train.append(l)

y_train = np.array(new_y_train)


In [ ]:
x_train.shape

In [ ]:
y_train.shape

Now we will make a helper class for the difference activation functions

In [ ]:
class Activation:
  def sigmoid(x):
    return 1/(1+np.exp(-x))
  def tanh(x):
    return np.tanh(x)
  def relu(x):
    return np.maximum(x,0)
  def sigmoid_der(x):
    return x*(1-x)
  def tanh_der(x):
    return 1-np.power(np.tanh(x), 2)
  def relu_der(x):
    return np.where(x >= 0, 1, 0)

Helper metrics class

In [ ]:
class Metrics:
  def __init__(self, y_pred, y_test):
    self.y_pred = y_pred
    self.y_actual = y_test
    self.len = len(y_pred)

  def accuracy(self):
    # Count the number of matching elements
    equals = np.sum(np.array_equal(self.y_pred, self.y_actual))
    return equals/self.len

  def mean_square_error(self):
    return np.mean(np.power(self.y_pred - self.y_actual, 2))
  


# Making the layers for the library

It's properties are:
 1. It takes an input from a previous layer 
 2. It gives and output to the next layer
 3. It computes the output through forward propogation
 4. It updates it's parameters through back propogation

In [ ]:
class Layer:
  def __init__(self):
    self.input = None
    self.output = None
  
  def foward_propogate(self, input):
    raise NotImplemenetedError
  
  def backward_propogate(self, output_error, lr):
    raise NotImplementedError

Let us define a Dense Layer (where every neuron is connected to every other neuron in the next layer) which is a subclass of our Layer class.

In [ ]:
class DenseLayer(Layer):
  # input_size = no of input neurons
  # output_size = no of output neurons
  def __init__(self, input_size, output_size, activation):
    # weights is an array of input_size rows and output_size columns
    # i-th row j-th column is the weight connecting i-th output neuron to j-th input neuron
    self.weights = np.zeros(input_size, output_size)
    # Apart from these there must be a bias neuron that is connected to all the output neurons
    self.bias = np.zeros(1,output_size)
    # Activation Function for this layer
    self.activation = activation
    # Input values, activation (weighted avg of input), outputs (activation function applied on activations), errors (of each neuron)
    self.inputs = np.zeros(1, input_size)
    self.activations = np.zeros(1, output_size)
    self.outputs = np.zeros(1, output_size)
    self.input_errors = np.zeros(1, input_size)

  # Given the input calculate the outputs (using the weights)
  # Input is a numpy array of 1 rows and input_size columns
  # Output is a numpy array of 1 rows and output_size columns
  def forward_propogate(self, inputs):
    self.inputs = inputs
    self.activations = np.dot(self.inputs, self.weights) + self.bias
    ac = Activation()
    match self.activation:
      case "relu":
        self.output = ac.relu(self.activation)
      case "sigmoid":
        self.output = ac.relu(self.activation)
      case "tanh":
        self.output = ac.relu(self.activation)
    return self.output
  
  # output_errors is a numpy array with 1 rows and output_size columns
  def backward_propogate(self, output_errors, lr):
    ac = Activation()
    # backpropogate the errors
    match self.activation:
      case "relu":
        self.input_errors = ac.relu_der(self.activation)*(np.dot(self.weights, output_errors.T)).T
      case "sigmoid":
        self.input_errors = ac.sigmoid_der(self.activation)*(np.dot(self.weights, output_errors.T)).T
      case "tanh":
        self.input_errors = ac.tanh_der(self.activation)*(np.dot(self.weights, output_errors.T)).T
    # Calculate the error in weights for this layer 
    weight_errors = np.dot(self.inputs.T, self.output_errors)
    # Update the weights
    self.weights -= lr*weight_errors
    # Update the bias
    self.bias -= lr*output_errors


# Neural Network Class

Will have all our dense layers (and the initial layer and final layer)

In [ ]:
class NeuralNetwork:
  def __init__(self):
    self.layers = []
    self.loss = None

  # Add a layer to our model
  def add(self, layer):
    self.layers.append(layer)
    return
  
  # Predict Output (through forward propogation)
  def predict(self, input_data):
    y_pred = []
    # Run the neural network over all the samples (stochastic gradient descent)
    for i in range(len(input_data)):
      # Forward propogate the values
      output = input_data[i]
      for layer in self.layers:
        output = layer.forward_propogate(output)
      y_pred.append(output)
    return y_pred

  # Train the network
  def fit(self, x_train, y_train, epochs=10, lr=0.001):
    self.loss = []
    for i in range(epochs):
      errors = 0
      for j in range(len(x_train)):
        # Forward propogate the values
        output = x_train[i]
        for layer in self.layers:
          output = layer.forward_propogate(output)
        errors = output - y_train[i]
        # Store the loss
        metrics = Metrics(output, y_train[i])
        self.loss = metrics.mean_square_error()
        # Backpropogate the errors
        for layer in reversed(self.layers):
          errors = layer.backward_propogate(errors, epochs, lr)